In [1]:
# Equalized-payout allocation for N mutually exclusive "No" contracts
# Each "No" contract on candidate i pays $1 if candidate i does NOT win.
# In any winning world (exactly one candidate wins), exactly (N-1) of the N "No" contracts pay $1.
# Provide either a total budget OR a target payout (per winning world).

from typing import List, Optional

def equalize_n_way_no(
    no_prices: List[float],
    budget: Optional[float] = None,
    target_payout: Optional[float] = None,  # desired payout in any winning world
    names: Optional[List[str]] = None,
):
    """
    Compute how many 'No' shares to buy for each of N candidates so the outcome is the same no matter who wins.

    Inputs:
      - no_prices: list of N floats (price of $1 'No' payout for each candidate), N >= 2.
      - budget: total dollars to spend (optional).
      - target_payout: dollars you want to receive in any winning world (optional).
      - names: optional list of N strings for display.

    Exactly one of budget or target_payout must be provided.

    Mechanics:
      - If you hold s shares of 'No' on each candidate equally, then in any world where one candidate wins,
        the other (N-1) 'No' positions pay out (N-1) * s dollars total.
      - Total spend = s * sum(no_prices).
      - Guaranteed profit condition (ignoring fees/slippage): sum(no_prices) < (N - 1).
    """
    N = len(no_prices)
    if N < 2:
        raise ValueError("Provide at least 2 'No' prices.")
    if (budget is None) == (target_payout is None):
        raise ValueError("Specify exactly one of: budget OR target_payout.")
    no_prices = [float(p) for p in no_prices]
    if any(p < 0 for p in no_prices):
        raise ValueError("All 'No' prices must be >= 0.")
    S = sum(no_prices)
    if S <= 0:
        raise ValueError("Sum of 'No' prices must be > 0.")

    # Equal shares across all N 'No' contracts equalizes outcomes.
    if target_payout is None:
        shares_each = budget / S
        total_spend = float(budget)
        payout_if_winner = (N - 1) * shares_each
    else:
        shares_each = float(target_payout) / (N - 1)
        total_spend = shares_each * S
        payout_if_winner = float(target_payout)

    spend_per_contract = [shares_each * p for p in no_prices]
    pnl_when_any_wins = payout_if_winner - total_spend
    roi = pnl_when_any_wins / total_spend if total_spend != 0 else float("nan")

    if names is None:
        names = [f"Candidate {i+1}" for i in range(N)]
    if len(names) != N:
        raise ValueError("Length of names must match number of prices.")

    rows = []
    for name, p, spend in zip(names, no_prices, spend_per_contract):
        rows.append({"candidate": name, "no_price": p, "spend": spend})

    return {
        "N": N,
        "shares_each": shares_each,
        "no_price_sum": S,
        "spend_per_contract": rows,
        "total_spend": total_spend,
        "payout_if_any_wins": payout_if_winner,
        "guaranteed_pnl": pnl_when_any_wins,
        "roi": roi,
        "is_arbitrage": S < (N - 1) - 1e-12,
    }


# --- Example usage ---
# Enter your N 'No' prices and either a budget OR a target payout (per winning world).
no_prices = [0.72, 0.69, 0.81, 0.88, 0.55]  # replace with live 'No' prices; any length N >= 2
budget = 100.0                              # total dollars to spend; comment this if using target_payout
# target_payout = 120.0                     # uncomment and set this instead of budget if you prefer

result = equalize_n_way_no(
    no_prices=no_prices,
    budget=budget,
    # target_payout=target_payout,
    names=[f"C{i+1}" for i in range(len(no_prices))],
)

print(f"N: {result['N']}")
print(f"Sum of 'No' prices: {result['no_price_sum']:.4f}")
print(f"Buy {result['shares_each']:.6f} 'No' shares of each contract (equal across all {result['N']}).")
for row in result["spend_per_contract"]:
    print(f"  {row['candidate']}: no_price={row['no_price']:.4f}, spend=${row['spend']:.2f}")
print(f"Total spend: ${result['total_spend']:.2f}")
print(f"Payout in any winning world ((N-1) 'No's paying): ${result['payout_if_any_wins']:.2f}")
print(f"Guaranteed P&L (any winner): ${result['guaranteed_pnl']:.2f} | ROI: {result['roi']*100:.2f}%")
print("Arbitrage opportunity!" if result["is_arbitrage"] else "No arbitrage (sum of 'No' prices >= N-1).")

N: 5
Sum of 'No' prices: 3.6500
Buy 27.397260 'No' shares of each contract (equal across all 5).
  C1: no_price=0.7200, spend=$19.73
  C2: no_price=0.6900, spend=$18.90
  C3: no_price=0.8100, spend=$22.19
  C4: no_price=0.8800, spend=$24.11
  C5: no_price=0.5500, spend=$15.07
Total spend: $100.00
Payout in any winning world ((N-1) 'No's paying): $109.59
Guaranteed P&L (any winner): $9.59 | ROI: 9.59%
Arbitrage opportunity!


In [2]:
equalize_n_way_no(no_prices=[0.27,0.99,0.91,0.77], budget=100)

{'N': 4,
 'shares_each': 34.013605442176875,
 'no_price_sum': 2.94,
 'spend_per_contract': [{'candidate': 'Candidate 1',
   'no_price': 0.27,
   'spend': 9.183673469387756},
  {'candidate': 'Candidate 2', 'no_price': 0.99, 'spend': 33.673469387755105},
  {'candidate': 'Candidate 3', 'no_price': 0.91, 'spend': 30.952380952380956},
  {'candidate': 'Candidate 4', 'no_price': 0.77, 'spend': 26.190476190476193}],
 'total_spend': 100.0,
 'payout_if_any_wins': 102.04081632653063,
 'guaranteed_pnl': 2.0408163265306314,
 'roi': 0.020408163265306315,
 'is_arbitrage': True}

In [3]:
equalize_n_way_no([1-0.55,1-0.26,1-0.12, 1-0.07], budget=100)

{'N': 4,
 'shares_each': 33.333333333333336,
 'no_price_sum': 3.0,
 'spend_per_contract': [{'candidate': 'Candidate 1',
   'no_price': 0.44999999999999996,
   'spend': 15.0},
  {'candidate': 'Candidate 2', 'no_price': 0.74, 'spend': 24.666666666666668},
  {'candidate': 'Candidate 3', 'no_price': 0.88, 'spend': 29.333333333333336},
  {'candidate': 'Candidate 4', 'no_price': 0.9299999999999999, 'spend': 31.0}],
 'total_spend': 100.0,
 'payout_if_any_wins': 100.0,
 'guaranteed_pnl': 0.0,
 'roi': 0.0,
 'is_arbitrage': False}

In [5]:
from polymarket_api import *

superbowl_singer = fetch_markets_by_keyword("Nothing Ever Happens: Taylor Swift Edition")
superbowl_singer

[{'id': '516719',
  'question': 'Russia x Ukraine ceasefire in 2025?',
  'slug': 'russia-x-ukraine-ceasefire-in-2025',
  'conditionId': '0x8ee2f1640386310eb5e7ffa596ba9335f2d324e303d21b0dfea6998874445791',
  'volumeNum': 20023745.910955,
  'outcomes': ['Yes', 'No'],
  'current_yes_prices': [0.115, 0.885],
  'current_no_prices_est': [0.885, 0.11499999999999999],
  'tokens_by_outcome': {'Yes': '15974786252393396629980467963784550802583781222733347534844974829144359265969',
   'No': '42351579455604873042957090441032286075880144797259231030267680078160718556927'},
  'search_score': 1.0,
  'search_matched': ['ever'],
  'raw': {'id': '516719',
   'question': 'Russia x Ukraine ceasefire in 2025?',
   'conditionId': '0x8ee2f1640386310eb5e7ffa596ba9335f2d324e303d21b0dfea6998874445791',
   'slug': 'russia-x-ukraine-ceasefire-in-2025',
   'resolutionSource': '',
   'endDate': '2025-12-31T12:00:00Z',
   'liquidity': '210092.7934',
   'startDate': '2024-12-29T22:51:33.18Z',
   'image': 'https://pol

In [5]:
ms = fetch_markets_by_keyword("halftime show")

In [6]:
len(ms)

18

In [7]:
from strategies import *

# Nothing happens military
# 
equalize_no_with_parlay([0.10,0.40,0.76,0.41],0.27, budget=100)

{'n': 4,
 'outcome_names': ['O1', 'O2', 'O3', 'O4'],
 'yes_prices': [0.1, 0.4, 0.76, 0.41],
 'no_prices': [0.9, 0.6, 0.24, 0.5900000000000001],
 'parlay_yes_price': 0.27,
 'shares_no_each': 32.6797385620915,
 'shares_no_by_outcome': {'O1': 32.6797385620915,
  'O2': 32.6797385620915,
  'O3': 32.6797385620915,
  'O4': 32.6797385620915},
 'share_parlay_yes': 0.0,
 'share_parlay_no': 32.6797385620915,
 'spend_no_by_outcome': {'O1': 29.41176470588235,
  'O2': 19.6078431372549,
  'O3': 7.84313725490196,
  'O4': 19.28104575163399},
 'spend_parlay_yes': 0.0,
 'spend_parlay_no': 23.856209150326794,
 'sum_no_prices': 2.33,
 'arbitrage_margin_per_share': 0.94,
 'arbitrage': True,
 'total_spend': 99.99999999999999,
 'payout': 130.718954248366,
 'guaranteed_payout': 130.718954248366,
 'guaranteed_profit': 30.718954248366018,
 'roi': 0.3071895424836602,
 'state_payouts': {'O1': 130.718954248366,
  'O2': 130.718954248366,
  'O3': 130.718954248366,
  'O4': 130.718954248366,
  'none_of_these': 130.7189

In [8]:
from strategies import *

equalize_no_with_parlay([0.55,0.26,0.12, 0.07],0.33, budget=100)

{'n': 4,
 'outcome_names': ['O1', 'O2', 'O3', 'O4'],
 'yes_prices': [0.55, 0.26, 0.12, 0.07],
 'no_prices': [0.44999999999999996, 0.74, 0.88, 0.9299999999999999],
 'parlay_yes_price': 0.33,
 'shares_no_each': 27.247956403269754,
 'shares_no_by_outcome': {'O1': 27.247956403269754,
  'O2': 27.247956403269754,
  'O3': 27.247956403269754,
  'O4': 27.247956403269754},
 'share_parlay_yes': 0.0,
 'share_parlay_no': 27.247956403269754,
 'spend_no_by_outcome': {'O1': 12.261580381471388,
  'O2': 20.16348773841962,
  'O3': 23.978201634877383,
  'O4': 25.34059945504087},
 'spend_parlay_yes': 0.0,
 'spend_parlay_no': 18.256130790190735,
 'sum_no_prices': 3.0,
 'arbitrage_margin_per_share': 0.33000000000000007,
 'arbitrage': True,
 'total_spend': 100.0,
 'payout': 108.99182561307902,
 'guaranteed_payout': 108.99182561307902,
 'guaranteed_profit': 8.991825613079016,
 'roi': 0.08991825613079016,
 'state_payouts': {'O1': 108.99182561307902,
  'O2': 108.99182561307902,
  'O3': 108.99182561307902,
  'O4'

In [1]:
from polymarket_api import get_market_and_description_by_url

# Example: pass the URL of the YES parlay market you’re analyzing
url = "https://polymarket.com/event/nothing-ever-happens-taylor-swift-edition?tid=1758908873696"  # replace with your actual parlay URL

market, description = get_market_and_description_by_url(
    url,
    active=True,     # optional filters
    closed=False,
)

if market is None:
    print("Market not found for URL (no exact slug or event slug match).")
else:
    print("Question:", market["question"])
    print("Slug:", market["slug"])
    # Check event slug(s) on the raw record for sanity
    ev_slugs = [ev.get("slug") for ev in (market["raw"].get("events") or []) if isinstance(ev, dict)]
    print("Event slugs:", ev_slugs)
    print("Outcomes:", market["outcomes"])
    print("Description preview:", (description or "")[:280], "...")


Market not found for URL (no exact slug or event slug match).


In [3]:
from polymarket_api import get_market_and_description_by_token_id

token_id = "1758908873696"  # example
market, description = get_market_and_description_by_token_id(token_id, active=True, closed=False)

if market is None:
    print("No market found for this token id.")
else:
    print(market["question"])

No market found for this token id.
